In [ ]:
import pandas as pd
import smote_variants as sv
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from Models.funciones import grafica_roc,classification_error
from xgboost import XGBClassifier #esta parcera toca instalarla pip install xgboost
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve
import scikitplot as skplt#Esta librería toca instalarla "conda install -c conda-forge scikit-plot"


import time

In [ ]:
df1=pd.read_csv('./CSV ready to model/withoutPCA.csv')
df2=pd.read_csv('./CSV ready to model/withoutPCA_minmax.csv')

Según el método wraper la mejor combinación de características es con 8 características, correspondientes
a (0, 1, 2, 3, 4, 6, 7, 8), de acá en adelante se van a hacer 4 modelos de XGBOOST
* 8 características y standar scaler
* 8 características y standar scaler
* 9 características y min max
* 9 características y min max

In [ ]:
le=LabelEncoder()
df1['GRAVEDAD']=df1.loc[:,['GRAVEDAD']].apply(le.fit_transform)
df2['GRAVEDAD']=df2.loc[:,['GRAVEDAD']].apply(le.fit_transform)

   ## Min max scaler
   

In [ ]:
y_minmax9=df2['GRAVEDAD']
x_minmax9=df2.drop(['GRAVEDAD','PERIODO'],axis=1)
y_minmax8=df2['GRAVEDAD']
x_minmax8=df2.drop(['GRAVEDAD','PERIODO','COMUNA'],axis=1)

In [ ]:
def model_XG(x,y,estimadores, caracteristicas, impresion = False,normalize=False):
    
    tiempo_i = time.time()
    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,3])
    recall_list = np.zeros([4,3])
    f_list = np.zeros([4,3])
    errores = np.zeros(4)
  
    XG = XGBClassifier(n_estimators=estimadores, max_features =caracteristicas, n_jobs = -1)
    for j in range(4):
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.25) # Modificar metodología de validación

        XG.fit(Xtrain, Ytrain)
        pred = XG.predict(Xtest)

        #code for calculating accuracy
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_

        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_


        errores[j] = classification_error(pred, Ytest)

    if impresion == True:
        skplt.metrics.plot_confusion_matrix(Ytest, pred)
        grafica_roc(model=XG,Xtest=Xtest,Ytest=Ytest)

    result={'Eficiencia':np.mean(accuracy_list),'Int_Eficiencia':np.std(accuracy_list),
            'Sensibilidad':np.mean(recall_list),'Int_Sensibilidad':np.std(recall_list),
            'Precisión':np.mean(precision_list),'Int Precisión':np.std(precision_list),
            'F':np.mean(f_list),'Int_F':np.std(f_list),
            'Error':np.mean(errores),'Int_Error':np.std(errores),
            'Tiempo':time.time()-tiempo_i
            }
    return  result

## Con 9 características

In [ ]:
result = model_XG(x=x_minmax9,y=y_minmax9,estimadores=100,caracteristicas= 8, impresion = True,normalize=False)

In [ ]:
print(result)

## Con 8 caracteristicas

In [ ]:
result2 = model_XG(x=x_minmax8,y=y_minmax8,estimadores=100,caracteristicas= 8, impresion = True,normalize=False)

In [ ]:
print(result2)

In [ ]:
result7 = model_XG(x=x_minmax8,y=y_minmax8,estimadores=50,caracteristicas= 8, impresion = True,normalize=False)

In [ ]:
print(result7)

In [ ]:
result8 = model_XG(x=x_minmax9,y=y_minmax9,estimadores=50,caracteristicas= 8, impresion = True,normalize=False)

In [ ]:
print(result8)

In [ ]:
# Standarscaler


In [ ]:
y_standar9=df1['GRAVEDAD']
x_standar9=df1.drop(['GRAVEDAD','PERIODO'],axis=1)
y_standar8=df1['GRAVEDAD']
x_standar8=df1.drop(['GRAVEDAD','PERIODO','COMUNA'],axis=1)

## Con 8 características

In [ ]:
result3 = model_XG(x=x_standar9,y=y_standar9,estimadores=100,caracteristicas= 8, impresion = True,normalize=False)

In [ ]:
print(result3)

## Con 9 caracteristicas

In [ ]:
result4 = model_XG(x=x_standar9,y=y_standar9,estimadores=100,caracteristicas= 8, impresion = True,normalize=False)

In [ ]:
print(result4)

In [ ]:
result5 = model_XG(x=x_standar8,y=y_standar8,estimadores=50,caracteristicas= 8, impresion = True,normalize=False)

In [ ]:
print(result5)

In [ ]:
result6 = model_XG(x=x_standar9,y=y_standar9,estimadores=50,caracteristicas= 8, impresion = True,normalize=False)

In [ ]:
print(result6)

In [ ]:
def imprimir_resultados(results,medida):
    for i in range(len(results)):
        print(i+1,results[i][medida])

In [ ]:
lista=[result,result2,result3,result4,result5,result6,result7,result8]
len(lista)

In [ ]:
imprimir_resultados(lista,'Eficiencia')

In [ ]:
imprimir_resultados(lista,'Precisión')

In [ ]:
XG = XGBClassifier(n_estimators=50, max_features =8, n_jobs = -1)

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x_standar9,y_standar9, test_size=0.25) # Modificar metodología de validación
XG.fit(Xtrain, Ytrain)

In [ ]:
pred = XG.predict(Xtest)

In [ ]:
_accuracy_ = accuracy_score(Ytest, pred, normalize=True)
_precisions_ = precision_score(Ytest, pred, average=None)


In [ ]:
print(_accuracy_,np.mean(_precisions_))

In [ ]:
skplt.metrics.plot_confusion_matrix(Ytest, pred)
grafica_roc(model=XG,Xtest=Xtest,Ytest=Ytest)

In [ ]:
oversampler= sv.MulticlassOversampling(sv.distance_SMOTE(proportion=0.002))

In [ ]:
x_samp, y_samp= oversampler.sample(x_standar, y_standar)


In [ ]:
y_standar.count

In [ ]:
u,indices = np.unique(y_samp,return_counts = True)


In [ ]:
indices

In [ ]:
result=model_VC(X_samp,y_samp,impresion = True)

In [ ]:
y_samp.

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_samp, y_samp, test_size=0.25)


In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_samp, y_samp, test_size=0.25)


In [ ]:

vc1.fit(Xtrain,Ytrain)
vc2.fit(Xtrain,Ytrain)

pred1=vc1.predict(Xtest)
pred2=vc2.predict(Xtest)

skplt.metrics.plot_confusion_matrix(Ytest, pred1,normalize=True)
